In [1]:
# Encoder 

import torch
import torch.nn as nn
import torch.nn.functional as F
from src.layers import ResnetBlockFC
from torch_scatter import scatter_mean, scatter_max
from src.common import coordinate2index, normalize_coordinate, normalize_3d_coordinate, map2local
from src.encoder.unet import UNet
from src.encoder.unet3d import UNet3D


class LocalPoolPointnet(nn.Module):
    ''' PointNet-based encoder network with ResNet blocks for each point.
        Number of input points are fixed.
    
    Args:
        c_dim (int): dimension of latent code c
        dim (int): input points dimension
        hidden_dim (int): hidden dimension of the network
        scatter_type (str): feature aggregation when doing local pooling
        unet (bool): weather to use U-Net
        unet_kwargs (str): U-Net parameters
        unet3d (bool): weather to use 3D U-Net
        unet3d_kwargs (str): 3D U-Net parameters
        plane_resolution (int): defined resolution for plane feature
        grid_resolution (int): defined resolution for grid feature 
        plane_type (str): feature type, 'xz' - 1-plane, ['xz', 'xy', 'yz'] - 3-plane, ['grid'] - 3D grid volume
        padding (float): conventional padding paramter of ONet for unit cube, so [-0.5, 0.5] -> [-0.55, 0.55]
        n_blocks (int): number of blocks ResNetBlockFC layers
    '''

    def __init__(self, c_dim=128, dim=3, hidden_dim=128, scatter_type='max', 
                 unet=False, unet_kwargs=None, unet3d=False, unet3d_kwargs=None, 
                 plane_resolution=None, grid_resolution=None, plane_type='xz', padding=0.1, n_blocks=5):
        super().__init__()
        self.c_dim = c_dim

        self.fc_pos = nn.Linear(dim, 2*hidden_dim)
        self.blocks = nn.ModuleList([
            ResnetBlockFC(2*hidden_dim, hidden_dim) for i in range(n_blocks)
        ])
        self.fc_c = nn.Linear(hidden_dim, c_dim)

        self.actvn = nn.ReLU()
        self.hidden_dim = hidden_dim

        if unet:
            self.unet = UNet(c_dim, in_channels=c_dim, **unet_kwargs)
        else:
            self.unet = None

        if unet3d:
            self.unet3d = UNet3D(**unet3d_kwargs)
        else:
            self.unet3d = None

        self.reso_plane = plane_resolution
        self.reso_grid = grid_resolution
        self.plane_type = plane_type
        self.padding = padding

        if scatter_type == 'max':
            self.scatter = scatter_max
        elif scatter_type == 'mean':
            self.scatter = scatter_mean
        else:
            raise ValueError('incorrect scatter type')


    def generate_plane_features(self, p, c, plane='xz'):
        # acquire indices of features in plane
        xy = normalize_coordinate(p.clone(), plane=plane, padding=self.padding) # normalize to the range of (0, 1)
        index = coordinate2index(xy, self.reso_plane)

        # scatter plane features from points
        fea_plane = c.new_zeros(p.size(0), self.c_dim, self.reso_plane**2)
        c = c.permute(0, 2, 1) # B x 512 x T
        fea_plane = scatter_mean(c, index, out=fea_plane) # B x 512 x reso^2
        fea_plane = fea_plane.reshape(p.size(0), self.c_dim, self.reso_plane, self.reso_plane) # sparce matrix (B x 512 x reso x reso)

        # process the plane features with UNet
        if self.unet is not None:
            fea_plane = self.unet(fea_plane)

        return fea_plane

    def generate_grid_features(self, p, c):
        p_nor = normalize_3d_coordinate(p.clone(), padding=self.padding)
        index = coordinate2index(p_nor, self.reso_grid, coord_type='3d')
        # scatter grid features from points
        fea_grid = c.new_zeros(p.size(0), self.c_dim, self.reso_grid**3)
        c = c.permute(0, 2, 1)
        fea_grid = scatter_mean(c, index, out=fea_grid) # B x C x reso^3
        fea_grid = fea_grid.reshape(p.size(0), self.c_dim, self.reso_grid, self.reso_grid, self.reso_grid) # sparce matrix (B x 512 x reso x reso)

        if self.unet3d is not None:
            fea_grid = self.unet3d(fea_grid)

        return fea_grid

    def pool_local(self, xy, index, c):
        bs, fea_dim = c.size(0), c.size(2)
        keys = xy.keys()

        c_out = 0
        for key in keys:
            # scatter plane features from points
            if key == 'grid':
                fea = self.scatter(c.permute(0, 2, 1), index[key], dim_size=self.reso_grid**3)
            else:
                fea = self.scatter(c.permute(0, 2, 1), index[key], dim_size=self.reso_plane**2)
            if self.scatter == scatter_max:
                fea = fea[0]
            # gather feature back to points
            fea = fea.gather(dim=2, index=index[key].expand(-1, fea_dim, -1))
            c_out += fea
        return c_out.permute(0, 2, 1)


    def forward(self, p):
        batch_size, T, D = p.size()

        # acquire the index for each point
        coord = {}
        index = {}
        if 'xz' in self.plane_type:
            coord['xz'] = normalize_coordinate(p.clone(), plane='xz', padding=self.padding)
            index['xz'] = coordinate2index(coord['xz'], self.reso_plane)
        if 'xy' in self.plane_type:
            coord['xy'] = normalize_coordinate(p.clone(), plane='xy', padding=self.padding)
            index['xy'] = coordinate2index(coord['xy'], self.reso_plane)
        if 'yz' in self.plane_type:
            coord['yz'] = normalize_coordinate(p.clone(), plane='yz', padding=self.padding)
            index['yz'] = coordinate2index(coord['yz'], self.reso_plane)
        if 'grid' in self.plane_type:
            coord['grid'] = normalize_3d_coordinate(p.clone(), padding=self.padding)
            index['grid'] = coordinate2index(coord['grid'], self.reso_grid, coord_type='3d')
        
        net = self.fc_pos(p)

        net = self.blocks[0](net)
        for block in self.blocks[1:]:
            pooled = self.pool_local(coord, index, net)
            net = torch.cat([net, pooled], dim=2)
            net = block(net)

        c = self.fc_c(net)

        fea = {}
        if 'grid' in self.plane_type:
            fea['grid'] = self.generate_grid_features(p, c)
        if 'xz' in self.plane_type:
            fea['xz'] = self.generate_plane_features(p, c, plane='xz')
        if 'xy' in self.plane_type:
            fea['xy'] = self.generate_plane_features(p, c, plane='xy')
        if 'yz' in self.plane_type:
            fea['yz'] = self.generate_plane_features(p, c, plane='yz')

        return fea


In [2]:
# Decoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from src.layers import ResnetBlockFC
from src.common import normalize_coordinate, normalize_3d_coordinate, map2local


class LocalDecoder(nn.Module):
    ''' Decoder.
        Instead of conditioning on global features, on plane/volume local features.

    Args:
        dim (int): input dimension
        c_dim (int): dimension of latent conditioned code c
        hidden_size (int): hidden size of Decoder network
        n_blocks (int): number of blocks ResNetBlockFC layers
        leaky (bool): whether to use leaky ReLUs
        sample_mode (str): sampling feature strategy, bilinear|nearest
        padding (float): conventional padding paramter of ONet for unit cube, so [-0.5, 0.5] -> [-0.55, 0.55]
    '''

    def __init__(self, dim=3, c_dim=128,
                 hidden_size=256, n_blocks=5, leaky=False, sample_mode='bilinear', padding=0.1):
        super().__init__()
        self.c_dim = c_dim
        self.n_blocks = n_blocks

        if c_dim != 0:
            self.fc_c = nn.ModuleList([
                nn.Linear(c_dim, hidden_size) for i in range(n_blocks)
            ])


        self.fc_p = nn.Linear(dim, hidden_size)

        self.blocks = nn.ModuleList([
            ResnetBlockFC(hidden_size) for i in range(n_blocks)
        ])

        self.fc_out = nn.Linear(hidden_size, 1)

        if not leaky:
            self.actvn = F.relu
        else:
            self.actvn = lambda x: F.leaky_relu(x, 0.2)

        self.sample_mode = sample_mode
        self.padding = padding
    

    def sample_plane_feature(self, p, c, plane='xz'):
        xy = normalize_coordinate(p.clone(), plane=plane, padding=self.padding) # normalize to the range of (0, 1)
        xy = xy[:, :, None].float()
        vgrid = 2.0 * xy - 1.0 # normalize to (-1, 1)
        c = F.grid_sample(c, vgrid, padding_mode='border', align_corners=True, mode=self.sample_mode).squeeze(-1)
        return c

    def sample_grid_feature(self, p, c):
        p_nor = normalize_3d_coordinate(p.clone(), padding=self.padding) # normalize to the range of (0, 1)
        p_nor = p_nor[:, :, None, None].float()
#         print(np.unique(p.clone().cpu().numpy()),np.unique(p_nor.cpu().numpy()))
        vgrid = 2.0 * p_nor - 1.0 # normalize to (-1, 1)
        # acutally trilinear interpolation if mode = 'bilinear'
        c = F.grid_sample(c, vgrid, padding_mode='border', align_corners=True, mode=self.sample_mode).squeeze(-1).squeeze(-1)
        return c


    def forward(self, p, c_plane, **kwargs):
        if self.c_dim != 0:
            plane_type = list(c_plane.keys())
            c = 0
            if 'grid' in plane_type:
                c += self.sample_grid_feature(p, c_plane['grid'])
            if 'xz' in plane_type:
                c += self.sample_plane_feature(p, c_plane['xz'], plane='xz')
            if 'xy' in plane_type:
                c += self.sample_plane_feature(p, c_plane['xy'], plane='xy')
            if 'yz' in plane_type:
                c += self.sample_plane_feature(p, c_plane['yz'], plane='yz')
            c = c.transpose(1, 2)

        p = p.float()
        net = self.fc_p(p)

        for i in range(self.n_blocks):
            if self.c_dim != 0:
                net = net + self.fc_c[i](c)

            net = self.blocks[i](net)

        out = self.fc_out(self.actvn(net))
        out = out.squeeze(-1)

        return out


In [3]:
# CON

import torch
import torch.nn as nn
from torch import distributions as dist
# from src.conv_onet.models import decoder


class ConvolutionalOccupancyNetwork(nn.Module):
    ''' Occupancy Network class.

    Args:
        decoder (nn.Module): decoder network
        encoder (nn.Module): encoder network
        device (device): torch device
    '''

    def __init__(self, decoder, encoder=None, device=None):
        super().__init__()
        
        self.decoder = decoder.to(device)

        if encoder is not None:
            self.encoder = encoder.to(device)
        else:
            self.encoder = None

        self._device = device

    def forward(self, p, inputs, sample=True, **kwargs):
        ''' Performs a forward pass through the network.

        Args:
            p (tensor): sampled points
            inputs (tensor): conditioning input
            sample (bool): whether to sample for z
        '''
        #############
        if isinstance(p, dict):
            batch_size = p['p'].size(0)
        else:
            batch_size = p.size(0)
        c = self.encode_inputs(inputs)
        p_r = self.decode(p, c, **kwargs)
        return p_r

    def encode_inputs(self, inputs):
        ''' Encodes the input.

        Args:
            input (tensor): the input
        '''

        if self.encoder is not None:
            c = self.encoder(inputs)
        else:
            # Return inputs?
            c = torch.empty(inputs.size(0), 0)

        return c

    def decode(self, p, c, **kwargs):
        ''' Returns occupancy probabilities for the sampled points.

        Args:
            p (tensor): points
            c (tensor): latent conditioned code c
        '''

        logits = self.decoder(p, c, **kwargs)
        p_r = dist.Bernoulli(logits=logits)
        return p_r

    def to(self, device):
        ''' Puts the model to the device.

        Args:
            device (device): pytorch device
        '''
        model = super().to(device)
        model._device = device
        return model


In [4]:
class SubsamplePointcloud(object):
    ''' Point cloud subsampling transformation class.

    It subsamples the point cloud data.

    Args:
        N (int): number of points to be subsampled
    '''
    def __init__(self, N):
        self.N = N

    def __call__(self, data):
        ''' Calls the transformation.

        Args:
            data (dict): data dictionary
        '''
        data_out = data.copy()
        points = data[None]
        normals = data['normals']

        indices = np.random.randint(points.shape[0], size=self.N)
        data_out[None] = points[indices, :]
        data_out['normals'] = normals[indices, :]

        return data_out
    
    
class SubsamplePoints(object):
    ''' Points subsampling transformation class.

    It subsamples the points data.

    Args:
        N (int): number of points to be subsampled
    '''
    def __init__(self, N):
        self.N = N

    def __call__(self, data):
        ''' Calls the transformation.

        Args:
            data (dictionary): data dictionary
        '''

        points = data[None]
        occ = data['occ']

        data_out = data.copy()
        if isinstance(self.N, int):
            idx = np.random.randint(points.shape[0], size=self.N)
            data_out.update({
                None: points[idx, :],
                'occ':  occ[idx],
            })
#         else:
#             Nt_out, Nt_in = self.N
#             occ_binary = (occ >= 0.5)
#             points0 = points[~occ_binary]
#             points1 = points[occ_binary]

#             idx0 = np.random.randint(points0.shape[0], size=Nt_out)
#             idx1 = np.random.randint(points1.shape[0], size=Nt_in)

#             points0 = points0[idx0, :]
#             points1 = points1[idx1, :]
#             points = np.concatenate([points0, points1], axis=0)

#             occ0 = np.zeros(Nt_out, dtype=np.float32)
#             occ1 = np.ones(Nt_in, dtype=np.float32)
#             occ = np.concatenate([occ0, occ1], axis=0)

#             volume = occ_binary.sum() / len(occ_binary)
#             volume = volume.astype(np.float32)

#             data_out.update({
#                 None: points,
#                 'occ': occ,
#                 'volume': volume,
#             })
        return data_out

In [5]:
# Carla Dataloader 

# import os
# import numpy as np
# import random
# import json

# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset
# from torch.utils import data

# class CarlaDataset(Dataset):
#     """Carla Simulation Dataset for 3D mapping project
    
#     Access to the processed data, including evaluation labels predictions velodyne poses times
#     """
#     def __init__(self, directory,
#         device='cuda',
#         num_frames=1,
#         voxelize_input=False,
#         binary_counts=True,
#         random_flips=False,       
#         ):
#         '''Constructor.
#         Parameters:
#             directory: directory to the dataset
#         '''

#         self.voxelize_input = voxelize_input
#         self.binary_counts = binary_counts
#         self._directory = directory
#         self._num_frames = num_frames
#         self.device = device
#         self.random_flips = random_flips
        
#         self._scenes = sorted(os.listdir(self._directory))
#         self._scenes = [os.path.join(scene, "cartesian") for scene in self._scenes]

#         self._num_scenes = len(self._scenes)
#         self._num_frames_scene = []

#         self._velodyne_list = []
#         self._eval_labels = []
#         self._frames_list = []

#         for scene in self._scenes:
#             velodyne_dir = os.path.join(self._directory, scene, 'pointcloud')
#             eval_dir = os.path.join(self._directory, scene, 'points')
#             self._num_frames_scene.append(len(os.listdir(velodyne_dir)))
            
#             frames_list = [os.path.splitext(filename)[0] for filename in sorted(os.listdir(velodyne_dir))]
#             self._frames_list.extend(frames_list)

#             self._velodyne_list.extend([os.path.join(velodyne_dir, 'pointcloud_' + str(frame).zfill(6) +'.npz') for frame in range(len(frames_list))])
#             self._eval_labels.extend([os.path.join(eval_dir, 'points_' + str(frame).zfill(6) +'.npz') for frame in range(len(frames_list))])

#         self._cum_num_frames = np.cumsum(np.array(self._num_frames_scene) - self._num_frames + 1)
#     # Use all frames, if there is no data then zero pad
#     def __len__(self):
#         return sum(self._num_frames_scene)
    
#     def collate_fn(self, batch):
#         #input_batch = [bi[0] for bi in data]
# #         output_batch = [bi[1] for bi in data]
#         #return input_batch
        
#         batch = list(filter(lambda x: x is not None, batch))
#         return data.dataloader.default_collate(batch)

#     def get_file_path(self, idx):
#         print(self._frames_list[idx])

        
#     def __getitem__(self, idx):

#         pointcloud = np.load(self._velodyne_list[idx])
#         points = np.load(self._eval_labels[idx])
        
#         pc_field = {}
#         pc_field[None] = pointcloud['points']
#         pc_field['normals'] = pointcloud['normals']
        
#         pc_field_sub = SubsamplePointcloud(10000)
#         pc_field = pc_field_sub(pc_field)
        
#         points_field = {}
#         points_field[None] = points['points']
#         points_field['occ'] = points['occupancies']
        
#         points_field_sub = SubsamplePoints(2048)
#         points_field = points_field_sub(points_field)
        
# #         print(pc_field[None].shape, pc_field['normals'].shape)
# #         print(points_field[None].shape, points_field['occupancies'].shape)
        
#         fields = {}
#         data = {}
        
#         #for pc_field,points_field in carla_ds:

#         fields['points'] = points_field
        
#         if pc_field is not None:
#             fields['inputs'] = pc_field

#         for field_name, field in fields.items():
#             field_data = field
#             for k, v in field_data.items():
#                 if k is None:
#                     data[field_name] = v
#                 else:
#                     data['%s.%s' % (field_name, k)] = v
# #         print(data['points'].shape, data['points.occ'].shape, data['inputs'].shape)
#         return data

In [6]:
def unpack(compressed):
    ''' given a bit encoded voxel grid, make a normal voxel grid out of it.  '''
    uncompressed = np.zeros(compressed.shape[0] * 8, dtype=np.uint8)
    uncompressed[::8] = compressed[:] >> 7 & 1
    uncompressed[1::8] = compressed[:] >> 6 & 1
    uncompressed[2::8] = compressed[:] >> 5 & 1
    uncompressed[3::8] = compressed[:] >> 4 & 1
    uncompressed[4::8] = compressed[:] >> 3 & 1
    uncompressed[5::8] = compressed[:] >> 2 & 1
    uncompressed[6::8] = compressed[:] >> 1 & 1
    uncompressed[7::8] = compressed[:] & 1

    return uncompressed

In [7]:
# Add points along ray to grid
def ray_trace_batch(points, labels, sample_spacing):

    # Compute samples using array broadcasting
    vec_norms_temp = np.linalg.norm(points, axis=1)

    # filter out zero norms
    points = points[vec_norms_temp != 0]
    labels = labels[vec_norms_temp != 0]
    vec_norms = np.reshape(vec_norms_temp[vec_norms_temp != 0], (-1,1))
    # vec_norms = np.reshape(np.linalg.norm(points, axis=1), (-1, 1))

    vec_angles = points / vec_norms

    difs = np.reshape(np.arange(0.0, 100.0, sample_spacing), (1, -1, 1))
    difs = np.reshape(vec_angles, (-1, 1, 3)) * difs
    new_samples = np.reshape(points, (-1, 1, 3)) - difs
    # Create labels
    new_labels = np.zeros((new_samples.shape[0], new_samples.shape[1]), dtype=np.uint8)
    new_labels[:, 0] = labels
    new_labels = new_labels.reshape(-1)

    # Remove points with dist < 0
    vec_dists = new_samples / np.reshape(vec_angles, (-1, 1, 3))
    first_pts = vec_dists[:, 0, 0]
    good_samples = np.reshape(new_samples[vec_dists[:, :, 0] > 0], (-1, 3))
    good_labels = new_labels[vec_dists[:, :, 0].reshape(-1) > 0]

    return good_samples, good_labels

In [8]:
# rellis3d dataset
import os
import numpy as np
import random
import json
import copy

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils import data

class RellisDataset(Dataset):
    """Rellis3D Dataset for convOccn
    
    Access to the processed data, including evaluation labels predictions velodyne poses times
    """
    def __init__(self, directory,
        device='cuda',
        num_frames=1,
        voxelize_input=False,
        binary_counts=True,
        random_flips=False,   
        model_setting = 'train'
        ):
        '''Constructor.
        Parameters:
            directory: directory to the dataset
        '''

        self.voxelize_input = voxelize_input
        self.binary_counts = binary_counts
        self._directory = directory
        self._num_frames = num_frames
        self.device = device
        self.random_flips = random_flips
        self.sampling_dist = 1.5
        
#         self._scenes = sorted(os.listdir(self._directory))
        self._scenes = [ s for s in sorted(os.listdir(self._directory)) if s.isdigit() ]

        self._num_scenes = len(self._scenes)
        self._num_frames_scene = 0
        
        self._velodyne_list = []
        self._poses = []
        self._frames_list = []
        self._num_frames_by_scene = []
        
        split_dir = os.path.join(self._directory, "pt_"+model_setting+".lst")

        # Generate list of scenes and indices to iterate over
        self._scenes_list = []
        self._index_list = []

        with open(split_dir, 'r') as split_file:
            for line in split_file:
                image_path = line.split(' ')
                image_path_lst = image_path[0].split('/')
                scene_num = image_path_lst[0]
                frame_index = int(image_path_lst[2][0:6])
                self._scenes_list.append(scene_num)
                self._index_list.append(frame_index)
                
        for scene_id in range(self._num_scenes):
            scene_name = self._scenes[scene_id]
            velodyne_dir = os.path.join(self._directory, scene_name, 'velodyne')
            # Load all poses and frame indices regardless of mode
            self._poses.append(np.loadtxt(os.path.join(self._directory, scene_name, 'poses.txt')).reshape(-1, 12) )
            self._frames_list.append([os.path.splitext(filename)[0] for filename in sorted(os.listdir(velodyne_dir))])
            self._num_frames_by_scene.append(len(self._frames_list[scene_id]))
#             print("nf:", self._num_frames_by_scene)
            # PC inputs
            self._velodyne_list.append( [os.path.join(velodyne_dir, 
                str(frame).zfill(6)+'.bin') for frame in self._frames_list[scene_id]] )
        
        # Get number of frames to iterate over
        self._num_frames_scene = len(self._index_list)
            

#         for i in range(len(self._scenes_list)):
#             scene_name = self._scenes_list[i]
#             velodyne_dir = os.path.join(self._directory, scene_name, 'velodyne')
#             self._poses.append(np.loadtxt(os.path.join(self._directory, scene_name, 'poses.txt')).reshape(-1, 12) )
#             self._frames_list.append([os.path.splitext(filename)[0] for filename in sorted(os.listdir(velodyne_dir))])
#             self._num_frames_scene += 1
            
#             #self._velodyne_list.extend([os.path.join(velodyne_dir, str(self._index_list[i]).zfill(6) + '.bin')])
#             self._velodyne_list.append( [os.path.join(velodyne_dir, 
#                 str(frame).zfill(6)+'.bin') for frame in self._frames_list[i]] )


    def get_pose(self, scene_id, frame_id):
        pose = np.zeros((4, 4))
        pose[3, 3] = 1
        pose[:3, :4] = self._poses[scene_id][frame_id].reshape(3, 4)
        return pose
        
        
    # Use all frames, if there is no data then zero pad
    def __len__(self):
#         return sum(self._num_frames_scene)
        return self._num_frames_scene
    
    def collate_fn(self, batch):
        batch = list(filter(lambda x: x is not None, batch))
        return data.dataloader.default_collate(batch)

    
    def __getitem__(self, idx):
        
        scene_name  = self._scenes_list[idx]
        scene_id    = int(scene_name)       # Scene ID
        frame_id    = self._index_list[idx] # Frame ID in current scene ID
        
        points = []
        points_occ = []
        
        # pointcloud
        pointcloud = np.fromfile(self._velodyne_list[scene_id][frame_id],dtype=np.float32).reshape(-1,4)[:,:3]

        frame_range = frame_id # remove leading 0s
#         if frame_range > 20:
#             frame_range = 20
        frame_range = 1
        
        ego_pose = self.get_pose(scene_id, frame_id)
        to_ego   = np.linalg.inv(ego_pose)
        
        for i in range(frame_range):
            velodyne = np.fromfile(self._velodyne_list[scene_id][frame_id - i],dtype=np.float32).reshape(-1,4)
#             print('range{}, scene{}, frame{}'.format(i, scene_id, frame_id - i))
#             print('x:', self._velodyne_list[scene_id][frame_id - i])
            velodyne[:,3] = 1            
            
            to_world = self.get_pose(scene_id, frame_id - i)
            relative_pose = np.matmul(to_ego, to_world)
            
            points_t = velodyne[:,:3]
            points_occ_t = velodyne[:,3]
            
            if i == 0: # current frame
                points_t, points_occ_t = ray_trace_batch(points_t, points_occ_t, self.sampling_dist)
#                 print('rt:',points_t.shape, points_occ_t.shape)
#                 print(np.unique(points_occ_t, return_counts=True))
            points_t = np.dot(relative_pose[:3, :3], points_t.T).T + relative_pose[:3, 3] # transfrom to current frame coordiantes
            
            # filter points outside the voxel grid
            min_bound = [-25.6, -25.6, -2]
            max_bound = [25.6, 25.6, 1.2]
            grid_point_mask= np.all(
            (points_t < max_bound) & (points_t >= min_bound), axis=1)
            points_t = points_t[grid_point_mask,:]
            points_occ_t = points_occ_t[grid_point_mask]
            
            points.extend(points_t)
            points_occ.extend(points_occ_t)
        
        points = np.asarray(points)
        points_occ = np.asarray(points_occ)
#         print('points:', points_occ.shape)
#         print('occ:', np.sum(points_occ == 1))
#         pointcloud = np.fromfile(self._velodyne_list[idx],dtype=np.float32).reshape(-1,4)[:,:3]

#         points_occ = velodyne[:,3]
        
#         mask = points_occ != 0
#         points_occ = points_occ[mask]
# #         print(np.unique(points_occ, return_counts=True))
        
#         # points
#         min_bound = [-25.6, -25.6, -2]
#         max_bound = [25.6, 25.6, 1.2]
#         grid_size = [256, 256, 16]
#         coor_ranges = min_bound + max_bound
#         voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / grid_size[0], 
#                     abs(coor_ranges[4] - coor_ranges[1]) / grid_size[1],
#                     abs(coor_ranges[5] - coor_ranges[2]) / grid_size[2]]

#         x = np.linspace(min_bound[0], max_bound[0], num=int(grid_size[0])) + voxel_sizes[0] / 2
#         y = np.linspace(min_bound[1], max_bound[1], num=int(grid_size[1])) + voxel_sizes[1] / 2
#         z = np.linspace(min_bound[2], max_bound[2], num=int(grid_size[2])) + voxel_sizes[2] / 2
#         xv, yv, zv = np.meshgrid(x, y, z, indexing="ij")

#         points = np.stack((xv, yv, zv))
#         points = np.moveaxis(points,0,-1).reshape(-1,3)
#         points = points[mask]
        
#         # Filter points outside of voxel grid
#         grid_point_mask= np.all(
#             (pointcloud < max_bound) & (pointcloud >= min_bound), axis=1)
#         pointcloud = pointcloud[grid_point_mask, :]
        
        # normalize
#         pc_sums = pointcloud.sum(axis=1, keepdims=True)
#         pointcloud /= (2 * pc_sums)
        pointcloud /= (2*np.max(pointcloud))

#         pointcloud = np.linalg.norm(pointcloud, axis=1)
        
#         p_sums = points.sum(axis=1, keepdims=True)
#         points /= (2 * p_sums)
        points /= (2*np.max(points))
        #pointcloud /= (2*np.max(pointcloud))
        #points /= (2*np.max(points))
        
        
        
#         print('nm:',np.unique(pointcloud, return_counts=True))
#         print('p:',np.unique(pointcloud, return_counts=True))
        
#         pocc_t = copy.deepcopy(points_occ)
        
#         points_occ[pocc_t != 40] = 1
#         points_occ[pocc_t == 40] = 0
#         points_occ[points_occ == 40] = 0
#         points_occ[points_occ != 0 ] = 1
                
        pc_field = {}
        pc_field[None] = pointcloud
        pc_field['normals'] = np.zeros((pointcloud.shape[0],1))
    
        pc_field_sub = SubsamplePointcloud(10000) #10000, 131072
        pc_field = pc_field_sub(pc_field)
        

        # normalize points
        
        points_field = {}
        points_field[None] = points
        points_field['occ'] = points_occ
        
        points_field_sub = SubsamplePoints(2048) #2048, 131072
        points_field = points_field_sub(points_field)
#         print('occ_sample',np.sum(points_field['occ'] == 1))
        
    #         print(pc_field[None].shape, pc_field['normals'].shape)
    #         print(points_field[None].shape, points_field['occupancies'].shape)
        
        fields = {}
        data = {}
        
        #for pc_field,points_field in carla_ds:

        fields['points'] = points_field
        
        if pc_field is not None:
            fields['inputs'] = pc_field

        for field_name, field in fields.items():
            field_data = field
            for k, v in field_data.items():
                if k is None:
                    data[field_name] = v
                else:
                    data['%s.%s' % (field_name, k)] = v
#         print(data['points'].shape, data['points.occ'].shape, data['inputs'].shape)
        return data

In [9]:
# Dataloader
from torch.utils.data import DataLoader


r3d_dir = "/home/jason/rellis3d"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
carla_ds_train = RellisDataset(directory=r3d_dir, device=device, model_setting='train')
carla_ds_eval = RellisDataset(directory=r3d_dir, device=device, model_setting='val')

dataloader_train = DataLoader(carla_ds_train, batch_size=4, shuffle=True, collate_fn=carla_ds_train.collate_fn, num_workers=8)
dataloader_eval = DataLoader(carla_ds_eval, batch_size=1, shuffle=False, collate_fn=carla_ds_eval.collate_fn, num_workers=4)
# o1= next(iter(Dataloader))
#print(list(o1[0].keys()), list(o2[0].keys()))
# test = o2[0]
# nz = np.sum(test!=0)
# print(o2[0].shape, nz)


cuda


In [10]:
# Model (configs/pointcloud/room_grid32.yaml)
# src.conv_onet.config.py get_model

encoder_kwargs = {'hidden_dim': 32,
                  'plane_type': 'grid',
                  'grid_resolution': 64, #32
                  'unet3d': True,
                  'unet3d_kwargs': {
                    'num_levels': 4,#3
                    'f_maps': 32,
                    'in_channels': 32,
                    'out_channels': 32}}

decoder_kwargs = {
    'sample_mode': 'bilinear', # bilinear / nearest
    'hidden_size': 32 }

c_dim = 32
dim = 3
padding = 0.1
decoder = LocalDecoder(dim=dim, c_dim=c_dim, padding=padding,
        **decoder_kwargs)

encoder = LocalPoolPointnet(dim=dim, c_dim=c_dim, padding=padding,
            **encoder_kwargs)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ConvolutionalOccupancyNetwork(
        decoder, encoder, device=device
    )

print(model)



ConvolutionalOccupancyNetwork(
  (decoder): LocalDecoder(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (fc_p): Linear(in_features=3, out_features=32, bias=True)
    (blocks): ModuleList(
      (0): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32, out_features=32, bias=True)
        (actvn): ReLU()
      )
      (1): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32, out_features=32, bias=True)
        (actvn): ReLU()
      )
      (2): ResnetBlockFC(
        (fc_0): Linear(in_features=32, out_features=32, bias=True)
        (fc_1): Linear(in_features=32

In [11]:
# compute loss 

def compute_loss(data):
    ''' Computes the loss.

    Args:
        data (dict): data dictionary
    '''    
    p = data.get('points').to(device)
    occ = data.get('points.occ').to(device)
    inputs = data.get('inputs', torch.empty(p.size(0), 0)).to(device)

    if 'pointcloud_crop' in data.keys():
        # add pre-computed index
        inputs = add_key(inputs, data.get('inputs.ind'), 'points', 'index', device=device)
        inputs['mask'] = data.get('inputs.mask').to(device)
        # add pre-computed normalized coordinates
        p = add_key(p, data.get('points.normalized'), 'p', 'p_n', device=device)

    c = model.encode_inputs(inputs)

    kwargs = {}
    # General points
    logits = model.decode(p, c, **kwargs).logits
    loss_i = F.binary_cross_entropy_with_logits(
        logits, occ.float(), reduction='none')
    loss = loss_i.sum(-1).mean()
    return loss

In [12]:
# compute iou
def compute_iou(occ1, occ2):
    ''' Computes the Intersection over Union (IoU) value for two sets of
    occupancy values.

    Args:
        occ1 (tensor): first set of occupancy values
        occ2 (tensor): second set of occupancy values
    '''
    occ1 = np.asarray(occ1)
    occ2 = np.asarray(occ2)

    # Put all data in second dimension
    # Also works for 1-dimensional data
    if occ1.ndim >= 2:
        occ1 = occ1.reshape(occ1.shape[0], -1)
    if occ2.ndim >= 2:
        occ2 = occ2.reshape(occ2.shape[0], -1)

    # Convert to boolean values
    occ1 = (occ1 >= 0.5)
    occ2 = (occ2 >= 0.5)

    # Compute IOU
    area_union = (occ1 | occ2).astype(np.float32).sum(axis=-1)
    area_intersect = (occ1 & occ2).astype(np.float32).sum(axis=-1)
    
#     print(area_union, area_intersect)

    iou = (area_intersect / area_union)

    return iou

def eval_step(data):
    model.eval()
    

In [13]:

# Intersection, union for one frame
def iou_one_frame(pred, target, n_classes=23):
    pred = np.asarray(pred)
    target = np.asarray(target)
    pred = pred.reshape(-1)
    target = target.reshape(-1)
    intersection = np.zeros(n_classes)
    union = np.zeros(n_classes)

    for cls in range(n_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection[cls] = (pred_inds[target_inds]).sum() # Cast to long to prevent overflows
        union[cls] = pred_inds.sum() + target_inds.sum() - intersection[cls]
    return intersection, union

In [14]:
# test
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import shutil
import copy

save_dir = '/home/jason/convolutional_occupancy_networks_og/out/r3d_model/'
write_dir = '/home/jason/convolutional_occupancy_networks_og/out/r3d_sw/'
writer = SummaryWriter(write_dir)
seed = 42
# setup_seed(seed)
lr = 0.001
BETA1 = 0.9
BETA2 = 0.999
decayRate = 0.96
epoch_num = 10000
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(BETA1, BETA2))
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
device = 'cuda'
print_every = 50

it = 0
loss_kt = 0
iou_kt = 0
iou_og_kt = 0
acc_kt = 0
best_iou = 0

velo_dir = '/home/jason/rellis3d/00000/velodyne'
output_dir = '/home/jason/convolutional_occupancy_networks_og/out/r3d_out/'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
    
if os.path.exists(write_dir):
    shutil.rmtree(write_dir)

os.mkdir(output_dir)
os.mkdir(save_dir)
os.mkdir(write_dir)

for epoch in range(epoch_num):

    model.train()
    for batch in dataloader_train:

        it += 1

        # Training
        optimizer.zero_grad()
        loss = compute_loss(batch)
        loss.backward()
        optimizer.step()
        loss_kt += loss.item()
        
        with torch.no_grad():
            # acc
            points = batch['points'].to(device)
            points_occ = batch['points.occ'].to(device)
            inputs = batch['inputs'].to(device)
            
            p_out = model(points, inputs)

            occ = (points_occ >= 0.5).cpu().numpy()
            occ_hat = (p_out.probs >= 0.5).cpu().numpy()
            acc = np.sum(occ == occ_hat, axis = 1) / occ.shape[1]

            intersection, union = iou_one_frame(occ, occ_hat, 2)
            mIou = np.mean(intersection / union)
            iou_kt += mIou
            
            og_iou = compute_iou(occ, occ_hat).mean()
            iou_og_kt += og_iou
            
            acc_batch = np.mean(acc)
            acc_kt += acc_batch
            
            # Record
            writer.add_scalar('/Loss/Train', loss.item(), it)
            writer.add_scalar('/Accuracy/Train', acc_batch, it)
            writer.add_scalar('/mIoU/Train', mIou, it)
            writer.add_scalar('/OGmIoU/Train', og_iou, it)
                
        if it % print_every == 0 :
            print("[Epoch %02d] it=%03d, loss=%.4f, acc=%.4f, iou=%.4f, oiou=%.4f" % (epoch, it,loss_kt/print_every, acc_kt/print_every, iou_kt/print_every, iou_og_kt/print_every))
            loss_kt = 0
            iou_kt = 0
            iou_og_kt = 0
            acc_kt = 0
        
    my_lr_scheduler.step()
    
    
    # Evaluation
    model.eval()
    acc_batches = 0
    iou = 0
    iou_og = 0
    with torch.no_grad():
        for bt in dataloader_eval:
            points = bt['points'].to(device)
            points_occ = bt['points.occ'].to(device)
            inputs = bt['inputs'].to(device)

            p_out = model(points, inputs)

            occ = (points_occ >= 0.5).cpu().numpy()
            occ_hat = (p_out.probs >= 0.5).cpu().numpy()
            acc = np.sum(occ == occ_hat, axis = 1) / occ.shape[1]
            acc_batch = np.mean(acc)        
            acc_batches += acc_batch
            
            intersection, union = iou_one_frame(occ, occ_hat, 2)
            mIou = np.mean(intersection/union)
            iou += mIou
            
            omIou = compute_iou(occ, occ_hat).mean()
            iou_og += omIou
            
            # Record
            writer.add_scalar('/Accuracy/Val', acc_batch, it)
            writer.add_scalar('/mIoU/Val', mIou, it)
            writer.add_scalar('/OGmIoU/Val', omIou, it)

    acc_batches /= len(dataloader_eval)
    iou /= len(dataloader_eval)
    iou_og /= len(dataloader_eval)
    print("acc_eval:%.2f, iou_eval:%.4f, oiou_eval:%.4f" % (acc_batches, iou, iou_og))
    
    
#     if iou >= best_iou:
#         best_iou = iou
    torch.save(model.state_dict(), os.path.join(save_dir, "Epoch" + str(epoch) + ".pt"))

    # Generate output        

    # use frame 100 - 150:
    with torch.no_grad():

        out_dir_ep = os.path.join(output_dir, "Epoch" + str(epoch))

        if not os.path.exists(out_dir_ep):
            os.mkdir(out_dir_ep)

        for j in range(100,150):
    #                 # points
            min_bound = [-25.6, -25.6, -2]
            max_bound = [25.6, 25.6, 1.2]
    #                 grid_size = [256, 256, 16]
    #                 coor_ranges = min_bound + max_bound
    #                 voxel_sizes = [abs(coor_ranges[3] - coor_ranges[0]) / grid_size[0], 
    #                             abs(coor_ranges[4] - coor_ranges[1]) / grid_size[1],
    #                             abs(coor_ranges[5] - coor_ranges[2]) / grid_size[2]]

    #                 x = np.linspace(min_bound[0], max_bound[0], num=int(grid_size[0])) + voxel_sizes[0] / 2
    #                 y = np.linspace(min_bound[1], max_bound[1], num=int(grid_size[1])) + voxel_sizes[1] / 2
    #                 z = np.linspace(min_bound[2], max_bound[2], num=int(grid_size[2])) + voxel_sizes[2] / 2
    #                 xv, yv, zv = np.meshgrid(x, y, z, indexing="ij")

    #                 points_t = np.stack((xv, yv, zv))
    #                 points_t = np.moveaxis(points_t,0,-1).reshape(-1,3)
    #                 points_t /= (2*np.max(points_t))

            pc_dir = os.path.join(velo_dir, str(j).zfill(6) + '.bin')
            pc_t = np.fromfile(pc_dir,dtype=np.float32).reshape(-1,4)
            pc_t[:,3] = 1
            inputs_t = copy.deepcopy(pc_t[:,:3])  
            points_t = copy.deepcopy(pc_t[:,:3]) 
            points_occ_t = copy.deepcopy(pc_t[:,3])

            points_t, points_occ_t = ray_trace_batch(points_t, points_occ_t, 1.5)

            grid_point_mask= np.all(
                (inputs_t < max_bound) & (inputs_t >= min_bound), axis=1)
            
            grid_point_mask_p= np.all(
                (points_t < max_bound) & (points_t >= min_bound), axis=1)

            inputs_t = inputs_t[grid_point_mask, :]
            points_t = points_t[grid_point_mask_p, :]
            points_occ_t = points_occ_t[grid_point_mask_p]

            # normalize
            inputs_t /= (2*np.max(inputs_t))
            points_t /= (2*np.max(points_t))

            pc_field_t = {}
            pc_field_t[None] = inputs_t
            pc_field_t['normals'] = np.zeros((inputs_t.shape[0],1))

            pc_field_sub_t = SubsamplePointcloud(inputs_t.shape[0]) #10000, inputs_t.shape[0]
            pc_field_t = pc_field_sub_t(pc_field_t)   

            points_field_t = {}
            points_field_t[None] = points_t
            points_field_t['occ'] = points_occ_t
            points_field_sub_t = SubsamplePoints(points_occ_t.shape) #2048, points_splits[k].shape[0]
            points_field_t = points_field_sub_t(points_field_t)

            fields_t = {}
            data_t = {}

            #for pc_field,points_field in carla_ds:

            fields_t['points'] = points_field_t

            if pc_field_t is not None:
                fields_t['inputs'] = pc_field_t

            for field_name_t, field_t in fields_t.items(): 

                field_data_t = field_t
                for k, v in field_data_t.items():
                    if k is None:
                        data_t[field_name_t] = v
                    else:
                        data_t['%s.%s' % (field_name_t, k)] = v
            points_t = torch.tensor(data_t['points'], device=device)
            points_t = points_t[None,:]
            inputs_t = torch.tensor(data_t['inputs'], device=device)
            inputs_t = inputs_t[None,:]
            p_out_t = model(points_t, inputs_t)
            occ_hat_t = (p_out_t.probs >= 0.5).cpu().numpy()
            occ_hat_t.astype('uint8').tofile(os.path.join(out_dir_ep, str(j).zfill(6) + '.label'))

    #                 # sliding windows(1/8 scale):
    #                 scale = 8
    #                 points_splits = np.array_split(points_t, scale)
    #                 occ_hat_t = []
    #                 for k in range(scale):
    #                     inputs_t = np.fromfile(pc_dir,dtype=np.float32).reshape(-1,4)[:,:3]  

    #                     grid_point_mask= np.all(
    #                         (inputs_t < max_bound) & (inputs_t >= min_bound), axis=1)
    #                     inputs_t = inputs_t[grid_point_mask, :]

    #                     # normalize
    #                     inputs_t /= (2*np.max(inputs_t))

    #                     pc_field_t = {}
    #                     pc_field_t[None] = inputs_t
    #                     pc_field_t['normals'] = np.zeros((inputs_t.shape[0],1))


    #                     pc_field_sub_t = SubsamplePointcloud(inputs_t.shape[0]) #10000
    #                     pc_field_t = pc_field_sub_t(pc_field_t)   

    #                     points_field_t = {}
    #                     points_field_t[None] = points_splits[k]
    #                     points_field_t['occ'] = np.zeros((points_splits[k].shape[0],1))
    #                     points_field_sub_t = SubsamplePoints(points_splits[k].shape[0]) #2048
    #                     points_field_t = points_field_sub_t(points_field_t)

    #                     fields_t = {}
    #                     data_t = {}

    #                     #for pc_field,points_field in carla_ds:

    #                     fields_t['points'] = points_field_t

    #                     if pc_field_t is not None:
    #                         fields_t['inputs'] = pc_field_t

    #                     for field_name_t, field_t in fields_t.items(): 

    #                         field_data_t = field_t
    #                         for k, v in field_data_t.items():
    #                             if k is None:
    #                                 data_t[field_name_t] = v
    #                             else:
    #                                 data_t['%s.%s' % (field_name_t, k)] = v
    #                     points_t = torch.tensor(data_t['points'], device=device)
    #                     points_t = points_t[None,:]
    #                     inputs_t = torch.tensor(data_t['inputs'], device=device)
    #                     inputs_t = inputs_t[None,:]
    #                     p_out_t = model(points_t, inputs_t)
    #                     occ_hat_t = np.append(occ_hat_t, (p_out_t.probs >= 0.5).cpu().numpy())
    #                 occ_hat_t.astype('uint8').tofile(os.path.join(out_dir_ep, str(j).zfill(6) + '.label'))

[Epoch 00] it=050, loss=664.0649, acc=0.8800, iou=0.6114, oiou=0.3528
[Epoch 00] it=100, loss=602.3489, acc=0.8889, iou=0.6396, oiou=0.4020
[Epoch 00] it=150, loss=587.3477, acc=0.8916, iou=0.6522, oiou=0.4237
[Epoch 00] it=200, loss=574.2052, acc=0.8935, iou=0.6563, oiou=0.4291
[Epoch 00] it=250, loss=561.4185, acc=0.8960, iou=0.6652, oiou=0.4422
[Epoch 00] it=300, loss=539.0039, acc=0.8998, iou=0.6704, oiou=0.4509
[Epoch 00] it=350, loss=555.4183, acc=0.8960, iou=0.6746, oiou=0.4627
[Epoch 00] it=400, loss=541.5849, acc=0.8991, iou=0.6675, oiou=0.4439
[Epoch 00] it=450, loss=523.7927, acc=0.9017, iou=0.6831, oiou=0.4734
[Epoch 00] it=500, loss=507.1121, acc=0.9024, iou=0.6790, oiou=0.4650
[Epoch 00] it=550, loss=509.7968, acc=0.9026, iou=0.6893, oiou=0.4817
[Epoch 00] it=600, loss=499.5844, acc=0.9045, iou=0.6922, oiou=0.4862
[Epoch 00] it=650, loss=499.6782, acc=0.9033, iou=0.6917, oiou=0.4874
[Epoch 00] it=700, loss=489.5839, acc=0.9064, iou=0.6920, oiou=0.4834
[Epoch 00] it=750, l

[Epoch 02] it=5800, loss=333.4947, acc=0.9316, iou=0.7753, oiou=0.6232
[Epoch 02] it=5850, loss=331.1685, acc=0.9331, iou=0.7794, oiou=0.6295
acc_eval:0.91, iou_eval:0.6887, oiou_eval:0.4784
[Epoch 03] it=5900, loss=330.9755, acc=0.9329, iou=0.7739, oiou=0.6210
[Epoch 03] it=5950, loss=333.0868, acc=0.9320, iou=0.7748, oiou=0.6229
[Epoch 03] it=6000, loss=328.2603, acc=0.9329, iou=0.7776, oiou=0.6287
[Epoch 03] it=6050, loss=325.0927, acc=0.9332, iou=0.7777, oiou=0.6275
[Epoch 03] it=6100, loss=323.5969, acc=0.9338, iou=0.7823, oiou=0.6357
[Epoch 03] it=6150, loss=323.3886, acc=0.9337, iou=0.7796, oiou=0.6308
[Epoch 03] it=6200, loss=316.0515, acc=0.9356, iou=0.7863, oiou=0.6391
[Epoch 03] it=6250, loss=312.4378, acc=0.9368, iou=0.7908, oiou=0.6491
[Epoch 03] it=6300, loss=316.5647, acc=0.9353, iou=0.7855, oiou=0.6382
[Epoch 03] it=6350, loss=323.1212, acc=0.9341, iou=0.7819, oiou=0.6342
[Epoch 03] it=6400, loss=325.2757, acc=0.9336, iou=0.7746, oiou=0.6211
[Epoch 03] it=6450, loss=322

[Epoch 05] it=11450, loss=288.2378, acc=0.9406, iou=0.8031, oiou=0.6714
[Epoch 05] it=11500, loss=293.0748, acc=0.9395, iou=0.8007, oiou=0.6635
[Epoch 05] it=11550, loss=287.0733, acc=0.9404, iou=0.8037, oiou=0.6700
[Epoch 05] it=11600, loss=283.7447, acc=0.9410, iou=0.8022, oiou=0.6695
[Epoch 05] it=11650, loss=282.2583, acc=0.9420, iou=0.8067, oiou=0.6778
[Epoch 05] it=11700, loss=274.4565, acc=0.9440, iou=0.8111, oiou=0.6835
acc_eval:0.91, iou_eval:0.6969, oiou_eval:0.4947
[Epoch 06] it=11750, loss=276.2311, acc=0.9427, iou=0.8075, oiou=0.6761
[Epoch 06] it=11800, loss=288.2676, acc=0.9398, iou=0.7986, oiou=0.6621
[Epoch 06] it=11850, loss=283.8903, acc=0.9413, iou=0.8014, oiou=0.6680
[Epoch 06] it=11900, loss=287.8096, acc=0.9399, iou=0.7968, oiou=0.6598
[Epoch 06] it=11950, loss=286.9414, acc=0.9413, iou=0.8050, oiou=0.6744
[Epoch 06] it=12000, loss=293.5506, acc=0.9387, iou=0.7955, oiou=0.6575
[Epoch 06] it=12050, loss=273.1064, acc=0.9440, iou=0.8113, oiou=0.6841
[Epoch 06] it=1

[Epoch 08] it=17050, loss=260.7812, acc=0.9457, iou=0.8186, oiou=0.6963
[Epoch 08] it=17100, loss=269.5999, acc=0.9441, iou=0.8104, oiou=0.6815
[Epoch 08] it=17150, loss=270.4935, acc=0.9431, iou=0.8142, oiou=0.6899
[Epoch 08] it=17200, loss=260.8437, acc=0.9451, iou=0.8174, oiou=0.6947
[Epoch 08] it=17250, loss=260.3504, acc=0.9459, iou=0.8202, oiou=0.7002
[Epoch 08] it=17300, loss=256.7992, acc=0.9465, iou=0.8212, oiou=0.7017
[Epoch 08] it=17350, loss=264.4157, acc=0.9457, iou=0.8171, oiou=0.6947
[Epoch 08] it=17400, loss=268.6627, acc=0.9440, iou=0.8122, oiou=0.6862
[Epoch 08] it=17450, loss=256.6530, acc=0.9462, iou=0.8203, oiou=0.6987
[Epoch 08] it=17500, loss=263.6518, acc=0.9447, iou=0.8161, oiou=0.6896
[Epoch 08] it=17550, loss=254.6562, acc=0.9465, iou=0.8196, oiou=0.6961
acc_eval:0.91, iou_eval:0.7029, oiou_eval:0.5081
[Epoch 09] it=17600, loss=265.6956, acc=0.9441, iou=0.8116, oiou=0.6869
[Epoch 09] it=17650, loss=260.5240, acc=0.9450, iou=0.8163, oiou=0.6924
[Epoch 09] it=1

[Epoch 11] it=22650, loss=237.3021, acc=0.9506, iou=0.8351, oiou=0.7229
[Epoch 11] it=22700, loss=254.0711, acc=0.9461, iou=0.8188, oiou=0.6963
[Epoch 11] it=22750, loss=243.9120, acc=0.9488, iou=0.8239, oiou=0.7068
[Epoch 11] it=22800, loss=244.6135, acc=0.9489, iou=0.8283, oiou=0.7133
[Epoch 11] it=22850, loss=249.6235, acc=0.9471, iou=0.8205, oiou=0.6983
[Epoch 11] it=22900, loss=245.0471, acc=0.9488, iou=0.8276, oiou=0.7095
[Epoch 11] it=22950, loss=247.9749, acc=0.9482, iou=0.8253, oiou=0.7071
[Epoch 11] it=23000, loss=248.5014, acc=0.9478, iou=0.8255, oiou=0.7081
[Epoch 11] it=23050, loss=240.6293, acc=0.9493, iou=0.8286, oiou=0.7122
[Epoch 11] it=23100, loss=232.0808, acc=0.9521, iou=0.8415, oiou=0.7334
[Epoch 11] it=23150, loss=239.0403, acc=0.9500, iou=0.8331, oiou=0.7204
[Epoch 11] it=23200, loss=259.3435, acc=0.9452, iou=0.8196, oiou=0.6984
[Epoch 11] it=23250, loss=247.1781, acc=0.9480, iou=0.8249, oiou=0.7054
[Epoch 11] it=23300, loss=241.1955, acc=0.9494, iou=0.8314, oiou

...

[Epoch 54] it=106700, loss=185.8539, acc=0.9597, iou=0.8633, oiou=0.7726
[Epoch 54] it=106750, loss=189.3485, acc=0.9591, iou=0.8605, oiou=0.7687
[Epoch 54] it=106800, loss=193.8701, acc=0.9578, iou=0.8578, oiou=0.7630
[Epoch 54] it=106850, loss=199.7170, acc=0.9564, iou=0.8535, oiou=0.7568
[Epoch 54] it=106900, loss=189.3703, acc=0.9586, iou=0.8628, oiou=0.7701
[Epoch 54] it=106950, loss=186.2481, acc=0.9595, iou=0.8631, oiou=0.7727
[Epoch 54] it=107000, loss=195.1101, acc=0.9576, iou=0.8563, oiou=0.7594
[Epoch 54] it=107050, loss=195.3838, acc=0.9577, iou=0.8582, oiou=0.7632
[Epoch 54] it=107100, loss=189.9749, acc=0.9583, iou=0.8584, oiou=0.7653
[Epoch 54] it=107150, loss=201.3445, acc=0.9561, iou=0.8531, oiou=0.7553
[Epoch 54] it=107200, loss=195.6415, acc=0.9573, iou=0.8543, oiou=0.7564
[Epoch 54] it=107250, loss=189.4392, acc=0.9590, iou=0.8607, oiou=0.7682
acc_eval:0.91, iou_eval:0.7051, oiou_eval:0.5097
[Epoch 55] it=107300, loss=185.6277, acc=0.9595, iou=0.8622, oiou=0.7703
[E

[Epoch 57] it=112250, loss=190.7421, acc=0.9581, iou=0.8589, oiou=0.7662
[Epoch 57] it=112300, loss=186.2021, acc=0.9599, iou=0.8664, oiou=0.7784
[Epoch 57] it=112350, loss=190.0062, acc=0.9589, iou=0.8610, oiou=0.7685
[Epoch 57] it=112400, loss=180.4105, acc=0.9612, iou=0.8672, oiou=0.7788
[Epoch 57] it=112450, loss=191.7319, acc=0.9585, iou=0.8600, oiou=0.7670
[Epoch 57] it=112500, loss=185.8565, acc=0.9600, iou=0.8655, oiou=0.7751
[Epoch 57] it=112550, loss=190.8860, acc=0.9584, iou=0.8614, oiou=0.7696
[Epoch 57] it=112600, loss=185.7618, acc=0.9601, iou=0.8644, oiou=0.7727
[Epoch 57] it=112650, loss=189.3062, acc=0.9588, iou=0.8630, oiou=0.7709
[Epoch 57] it=112700, loss=194.1485, acc=0.9578, iou=0.8576, oiou=0.7605
[Epoch 57] it=112750, loss=182.0661, acc=0.9608, iou=0.8671, oiou=0.7773
[Epoch 57] it=112800, loss=180.9969, acc=0.9607, iou=0.8672, oiou=0.7774
[Epoch 57] it=112850, loss=186.0455, acc=0.9597, iou=0.8638, oiou=0.7721
[Epoch 57] it=112900, loss=185.6040, acc=0.9599, io

[Epoch 60] it=117800, loss=189.1811, acc=0.9583, iou=0.8606, oiou=0.7687
[Epoch 60] it=117850, loss=191.4680, acc=0.9585, iou=0.8621, oiou=0.7696
[Epoch 60] it=117900, loss=191.0013, acc=0.9586, iou=0.8598, oiou=0.7661
[Epoch 60] it=117950, loss=191.3557, acc=0.9587, iou=0.8614, oiou=0.7691
[Epoch 60] it=118000, loss=189.6686, acc=0.9586, iou=0.8595, oiou=0.7670
[Epoch 60] it=118050, loss=187.0806, acc=0.9596, iou=0.8632, oiou=0.7713
[Epoch 60] it=118100, loss=179.8138, acc=0.9613, iou=0.8686, oiou=0.7810
[Epoch 60] it=118150, loss=181.7948, acc=0.9606, iou=0.8676, oiou=0.7786
[Epoch 60] it=118200, loss=195.8859, acc=0.9573, iou=0.8575, oiou=0.7612
[Epoch 60] it=118250, loss=188.2849, acc=0.9594, iou=0.8633, oiou=0.7722
[Epoch 60] it=118300, loss=189.8420, acc=0.9590, iou=0.8614, oiou=0.7693
[Epoch 60] it=118350, loss=199.1585, acc=0.9562, iou=0.8506, oiou=0.7521
[Epoch 60] it=118400, loss=177.6541, acc=0.9618, iou=0.8720, oiou=0.7850
[Epoch 60] it=118450, loss=188.0995, acc=0.9594, io

[Epoch 63] it=123350, loss=170.2517, acc=0.9638, iou=0.8793, oiou=0.7964
[Epoch 63] it=123400, loss=194.5563, acc=0.9572, iou=0.8561, oiou=0.7611
[Epoch 63] it=123450, loss=186.9914, acc=0.9593, iou=0.8620, oiou=0.7714
[Epoch 63] it=123500, loss=185.4244, acc=0.9600, iou=0.8663, oiou=0.7756
[Epoch 63] it=123550, loss=186.1256, acc=0.9596, iou=0.8669, oiou=0.7794
[Epoch 63] it=123600, loss=194.7123, acc=0.9572, iou=0.8554, oiou=0.7587
[Epoch 63] it=123650, loss=192.0242, acc=0.9580, iou=0.8590, oiou=0.7659
[Epoch 63] it=123700, loss=195.7374, acc=0.9571, iou=0.8547, oiou=0.7571
[Epoch 63] it=123750, loss=189.0464, acc=0.9586, iou=0.8585, oiou=0.7634
[Epoch 63] it=123800, loss=183.4036, acc=0.9603, iou=0.8648, oiou=0.7740
[Epoch 63] it=123850, loss=191.2277, acc=0.9584, iou=0.8585, oiou=0.7636
[Epoch 63] it=123900, loss=189.3922, acc=0.9589, iou=0.8628, oiou=0.7725
[Epoch 63] it=123950, loss=180.7253, acc=0.9609, iou=0.8686, oiou=0.7811
[Epoch 63] it=124000, loss=184.3840, acc=0.9601, io

KeyboardInterrupt: 